In [1]:
%pip install requests

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: C:\Users\jyravi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [197]:
import requests
import json
import time


#### Your speech service region and the subscription_key

In [198]:
# Your subscription key and region for the speech service
SERVICE_REGION = ""

NAME = "Simple transcription"
DESCRIPTION = "Simple transcription description"

#LOCALE = "en-IN"
SUBSCRIPTION_KEY=" "

In [199]:
from http import HTTPStatus

expected_status_codes=[HTTPStatus.CREATED]
expected_status_codes

[<HTTPStatus.CREATED: 201>]

#### input audio in Azure blob and defining destination url for storing the transcribed file

In [152]:

destcontainerurl="" ### paste your Blob sas URL
uri=f"https://{SERVICE_REGION}.api.cognitive.microsoft.com/speechtotext/v3.2/transcriptions"

In [1]:
Sas_token=destcontainerurl.split("?")[-1]

##### diarization is possible only for mono audios.

In [203]:
content ={ "contentUrls": [
    
    "" # paste your url
  ],
  "locale": "en-IN",
  "displayName": "My Transcription",
  #"model": Null,
"properties": {
    #"wordLevelTimestampsEnabled": True,
    "diarizationEnabled": True,
    "timeToLive" : "PT30M",
    "destinationContainerUrl":destcontainerurl,
    "languageIdentification": {
      "candidateLocales": [
        "en-IN","hi-IN"
      ],
    }
  },}

In [204]:
headers = {"Ocp-Apim-Subscription-Key": SUBSCRIPTION_KEY}
response = requests.post(uri, headers=headers, json=content)
if response.status_code not in expected_status_codes :
    raise Exception(f"The POST request to {uri} returned a status code {response.status_code} that was not in the expected status codes: {expected_status_codes}")
else :
    try :
        response_json = response.json()
        print({ "headers" : response.headers, "text" : response.text, "json" : response_json })
    except Exception :
        { "headers" : response.headers, "text" : response.text, "json" : None }

{'headers': {'Cache-Control': 'no-store', 'Transfer-Encoding': 'chunked', 'Content-Type': 'application/json; charset=utf-8', 'Location': 'https://eastus.api.cognitive.microsoft.com/speechtotext/v3.2/transcriptions/e31b9863-c605-4023-be52-8ec8274cb48a', 'api-supported-versions': '2.0,3.0,3.1,3.2-preview2,3.2', 'api-deprecated-versions': '2.0,3.0', 'apim-request-id': '6c134555-797b-48fa-82b0-527816c3a2f1', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload', 'x-content-type-options': 'nosniff', 'x-ms-region': 'East US', 'Date': 'Fri, 13 Sep 2024 03:53:14 GMT'}, 'text': '{\n  "self": "https://eastus.api.cognitive.microsoft.com/speechtotext/v3.2/transcriptions/e31b9863-c605-4023-be52-8ec8274cb48a",\n  "links": {\n    "files": "https://eastus.api.cognitive.microsoft.com/speechtotext/v3.2/transcriptions/e31b9863-c605-4023-be52-8ec8274cb48a/files"\n  },\n  "properties": {\n    "diarizationEnabled": true,\n    "wordLevelTimestampsEnabled": false,\n    "displayFormWordLe

In [2]:
response_json

#### Get the status of the transcription

In [208]:
rr=requests.get(response_json['self'],headers=headers)

In [3]:
rr.json()

#### When 'status' == 'Succeeded'

In [4]:
response_json['links']['files']

#### Get the transcribed file

In [211]:
r=requests.get(response_json['links']['files'],headers=headers)

In [212]:
result_json=r.json()

In [5]:
result_json

In [6]:
result_json['values'][0]['links']['contentUrl']

#### Results extraction when destination url is not defined.

##### Here we have taken the only the first ContentUrl, as we have only given one URL. This will go into a loop when you pass multiple Urls as a list.

In [133]:
result_wo_sas=requests.get(result_json['values'][0]['links']['contentUrl'])

In [134]:
result_wo_sas.json()

{'source': 'https://www.uclass.psychol.ucl.ac.uk/Release2/Conversation/AudioOnly/wav/F_0101_10y4m_1.wav',
 'timestamp': '2024-09-12T08:46:56Z',
 'durationInTicks': 1377811250,
 'duration': 'PT2M17.781125S',
 'combinedRecognizedPhrases': [{'channel': 0,
   'lexical': "that's enough what happened what happened after she got out out of the she found the old building where she used to live and she went inside inside it and she started to have a look around yeah what does she find in the building diesel pictures on the wall and she found this old silver line case thing you forgotten about the puppy after she was thrown out a puppy came running oh boy two men they were trying to find what they were trying to sort of audition and they people to pretend to be her so they can get the money from the grandmother that's a great story yeah where are we",
   'itn': "that's enough what happened what happened after she got out out of the she found the old building where she used to live and she went i

### sas_token for the destination_url

In [215]:
Sas_token= 

#### constructing the url with sas_token for reading the file from destination url
##### Please create loop for reading through all the contentUrls if a list of URLs are provided for transcribing

In [7]:
file_uri=result_json['values'][0]['links']['contentUrl']+"?"+Sas_token
file_uri

In [217]:
result=requests.get(url=file_uri)

In [8]:
result.json()

In [219]:
result.json().keys()

dict_keys(['source', 'timestamp', 'durationInTicks', 'duration', 'combinedRecognizedPhrases', 'recognizedPhrases'])

#### Below step is one way to combine the speaker and the utterance. There could be other ways,please feel free to make those according how the the use wants the output to look like

In [220]:
conversation=[]
for i in result.json()['recognizedPhrases']:
    if i["recognitionStatus"]=='Success':
        #print('speaker '+ str(i['speaker'])+":")
        #print(i['nBest'][0]['lexical'])
        conversation.append(['speaker '+ str(i['speaker'])+": "+ i['nBest'][0]['lexical']])


In [221]:
conversation

[['speaker 1: good evening'],
 ['speaker 1: welcome to bhroove'],
 ['speaker 1: how can i help you'],
 ['speaker 2: sir sir'],
 ['speaker 2: बोल रहा हूँ'],
 ['speaker 1: हैलो हाँ सर बताइए क्या जानना है आपको'],
 ['speaker 2: मैने'],
 ['speaker 2: अभी मैं शाम में अपना अकाउन्ट बैलेंस देखा था तो सोलह सोलह सौ चौंसठ रुपए था बट अभी सिक्सटीन हंड्रेड फॉर्टी नाइन दिखा रहा'],
 ['speaker 1: है'],
 ['speaker 2: तो पंद्रह रुपए का काट'],
 ['speaker 2: thank you sir bataaenge'],
 ['speaker 1: check karo akhilesh pratap singh ka naam se'],
 ['speaker 1: रात को होती है चार्जेज वगैरह रात को डिडक्ट होते है अपने यहाँ पे'],
 ['speaker 1: h d f c sell kara ek aur h f sale yahaan pe trade kara to yahaan pe deposit'],
 ['speaker 1: आपको वॅन फाइव पॉइंट नाइन का रहेगा डी पी चार्जेज लगाया'],
 ['speaker 2: अच्छा'],
 ['speaker 1: तो'],
 ['speaker 2: एक रुपए डिडक्ट करना चाहिए ना तो कितना पंद्रह रुपए'],
 ['speaker 1: एक रुपए के होगा सर वॅन फाइव पॉइंट नाइन थ्री ये डेपॉज़िट चार्जेज पर कंपनी का ये आपको सी डी एस एल चार्ज 

### Passing "conversation" dynamically to AOAI.

##### Please make necessary changes in the system prompt to suit your use cases

In [231]:
import os
import requests
import base64

# Configuration
API_KEY = ""
headers = {
    "Content-Type": "application/json",
    "api-key": API_KEY,
}

# Payload for the request
payload = { "messages": [
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": '''You are AI assistant for analyzing transcripts between customer support agent and the customer. 
          You will be provided with the conversation transcripts with speakers diarized as speaker 1 and speaker 2.
          1. Identify agent and customer from the conversation flow.
          2.  Summarize the issue/query/concern from the customer?
          3. Summarize the response provided by the agent.
          4. Was the customer satisfied with the resolution provided by the agent?
          5.Overall sentiment at the end of the call.
          Please respond with the answers in the below json format
          "{"Speaker 1": "",
          "Speaker 2": " ",
          "Issue" : " ",
          "Resolution":"",
          "Customer Satified" : " ",
          "Overall Sentiment": " "}"'''
          
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": f"{conversation}"
        }
      ]
    },],  
  "temperature": 0.7,
  "top_p": 0.95,
  "max_tokens": 800}

ENDPOINT = f"https://{}.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-02-15-preview"



In [232]:
# Send request
try:
    response = requests.post(ENDPOINT, headers=headers, json=payload)
    response.raise_for_status()  # Will raise an HTTPError if the HTTP request returned an unsuccessful status code
except requests.RequestException as e:
    raise SystemExit(f"Failed to make the request. Error: {e}")

# Handle the response as needed (e.g., print or process)
print(response.json())

{'choices': [{'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}, 'finish_reason': 'stop', 'index': 0, 'logprobs': None, 'message': {'content': '```json\n{\n  "Speaker 1": "Agent",\n  "Speaker 2": "Customer",\n  "Issue": "The customer observed a deduction of Rs 15 from his account balance and wanted to understand the reason for this deduction.",\n  "Resolution": "The agent explained that the deduction was due to DP (Depository Participant) charges that are applied during the night settlement process. Specifically, a charge of Rs 15.93 was deducted as part of the company\'s CDS (Central Depository Services) charges.",\n  "Customer Satisfied": "Yes",\n  "Overall Sentiment": "Neutral to Positive"\n}\n```', 'role': 'assistant'}}], 'created': 1726200458, 'id': 'chatcmpl-A6rsoJEckj8sH70LGMGCFywI1bCrk', 'model': 

In [234]:
print(response.json()['choices'][0]['message']['content'])

```json
{
  "Speaker 1": "Agent",
  "Speaker 2": "Customer",
  "Issue": "The customer observed a deduction of Rs 15 from his account balance and wanted to understand the reason for this deduction.",
  "Resolution": "The agent explained that the deduction was due to DP (Depository Participant) charges that are applied during the night settlement process. Specifically, a charge of Rs 15.93 was deducted as part of the company's CDS (Central Depository Services) charges.",
  "Customer Satisfied": "Yes",
  "Overall Sentiment": "Neutral to Positive"
}
```
